In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../../')

from main import load_and_prepare_sessions
from analysis.timepoint_analysis import find_drug_split_x

import numpy as np
import config
from collections import defaultdict

sessions_cpt = load_and_prepare_sessions("../../../trial_Gq-DREADD_CPT", load_from_pickle=True, remove_bad_signal_sessions=True)
sessions_hab = load_and_prepare_sessions("../../../Gq-DREADD_CPT_Habituation", load_from_pickle=True, remove_bad_signal_sessions=True)

In [2]:
mouse_cpt_peaks = defaultdict(list)
mouse_hab_peaks = defaultdict(list)

for session in sessions_cpt:
    if session.drug_info['name'] != 'Vehicle':
        continue

    for br in session.brain_regions:
        mouse_cpt_peaks[session.mouse_id].\
            append(find_drug_split_x(session, br, find_relative_peak=True) + (br, ))
        

for session in sessions_hab:
    if session.mouse_id not in mouse_cpt_peaks.keys():
        continue
    for br in session.brain_regions:
        mouse_hab_peaks[session.mouse_id].\
            append(find_drug_split_x(session, br, find_relative_peak=True) + (br, ))

In [3]:
import pandas as pd
from collections import defaultdict

def dict_to_dataframe(data_dict, peak_type):
    df_list = []
    for mouse_id, entries in data_dict.items():
        for minutes, zf_score, brain_region in entries:
            df_list.append({
                'mouse_id': mouse_id,
                'minutes from cpt start': minutes,
                'zf-score': zf_score,
                'brain_region': brain_region,
                'type': peak_type
            })
    return pd.DataFrame(df_list)

# Convert both dicts to DataFrames and add the type
cpt_df = dict_to_dataframe(mouse_cpt_peaks, 'cpt')
hab_df = dict_to_dataframe(mouse_hab_peaks, 'hab')

# Combine both DataFrames
combined_df = pd.concat([cpt_df, hab_df])

# Set the MultiIndex with 'type' as the outermost, then 'mouse_id' and 'brain_region'
combined_df.set_index(['type', 'mouse_id', 'brain_region'], inplace=True)

# Save to Excel
combined_df.to_excel('mouse_peaks_2.xlsx')